# 1. Example of Linear Programming

In [1]:
!pip install ortools

**Basic steps for solving an LP problem**

To solve a LP problem, your program should include the following steps :

- Import the linear solver wrapper,
- declare the LP solver,
- define the variables,
- define the constraints,
- define the objective,
- call the LP solver; and
- display the solution

## A linear optimization example

Maximize 3*x* + *y* subject to the following constraints: 
```
0≤*x*≤1 ; 
0≤*y*≤2 ;
*x* + *y*≤2;
```

### Import the require libraries

In [1]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

### Declare the solver

In [2]:
# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver('GLOP')

pywraplp is a Python wrapper for the underlying C++ solver. The argument "GLOP" specifies GLOP, the OR-Tools linear solver.

**GLOP:** Google Linear Optimization. The primary OR-Tools linear optimization solver is Glop, Google's in-house linear programming solver. It's fast, memory efficient, and numerically stable.

### Create the variables

In [3]:
# Create the variables x and y.
x = solver.NumVar(0, 1, 'x')
y = solver.NumVar(0, 2, 'y')

print('Number of variables =', solver.NumVariables())

Number of variables = 2


### Define the constraints

The first two constraints, 0 ≤ x ≤ 1 and 0 ≤ y ≤ 2, are already set by the definitions of the variables. The following code defines the constraint x + y ≤ 2:

In [4]:
# Create a linear constraint, 0 <= x + y <= 2.
ct = solver.Constraint(0, 2, 'ct')
ct.SetCoefficient(x, 1)
ct.SetCoefficient(y, 1)

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 1


### Define the objective function

In [5]:
# Create the objective function, 3 * x + y.
objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 1)
objective.SetMaximization()

### Invoke the solver and display the results.


In [6]:
solver.Solve()
print('Solution:')
print('Objective value =', objective.Value())
print('x =', x.solution_value())
print('y =', y.solution_value())

Solution:
Objective value = 4.0
x = 1.0
y = 1.0


# 2. Examples of Constraint Programming

## Constraint optimization

Constraint optimization, or constraint programming (CP), identifies feasible solutions out of a very large set of candidates, where the problem can be modeled in terms of arbitrary constraints. **CP is based on feasibility (finding a feasible solution) rather than optimization (finding an optimal solution) and focuses on the constraints and variables rather than the objective function.** However, **CP can be used to solve optimization problems, simply by comparing the values of the objective function for all feasible solutions.**

### Tools

Google provides few ways to solve CP problems:

- [CP-SAT solver](https://developers.google.com/optimization/cp/cp_solver): A *constraint programming* solver that uses SAT (satisfiability) methods.
- [Original CP solver](https://developers.google.com/optimization/cp/original_cp_solver): A *constraint programming* solver.

If your problem can be modeled with a linear objective and linear constraints, then you have a [linear programming](https://developers.google.com/optimization/lp) problem and should consider [MPSolver](https://developers.google.com/optimization/lp/mpsolver). (However, routing problems are typically best solved with our [vehicle routing library](https://developers.google.com/optimization/reference/constraint_solver/routing) even if they can be represented with a linear model.)

## CP-SAT Solver

**§ Remark** : To increase computational speed, the CP-SAT solver works over the integers. This means you must define your optimization problem using integers only. If you begin with a problem that has constraints with non-integer terms, you need to first multiply those constraints by a sufficiently large integer so that all terms are integers.

### Example : finding a feasible solution

Let's start with a simple example problem in which there are:

**Three variables, x, y, and z, each of which can take on the values: 0, 1, or 2.**

**One constraint: x ≠ y**

We'll start by showing how to use the CP-SAT solver to find a single feasible solution in all of the supported languages. While finding a feasible solution is trivial in this case, in more complex constraint programming problems it can be very difficult to determine whether there is a feasible solution.

In [8]:
# Import the libraries
from ortools.sat.python import cp_model

# Declare the model
model = cp_model.CpModel()

# Create the variables

num_vals = 3
x = model.NewIntVar(0, num_vals - 1, 'x')
y = model.NewIntVar(0, num_vals - 1, 'y')
z = model.NewIntVar(0, num_vals - 1, 'z')

# Create the constraint
model.Add(x != y)

# Call the solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

### CP-SAT return values

The CP-SAT solver returns one of the status values shown in the table below. In this example, the value returned is `OPTIMAL`.

- OPTIMAL : An optimal feasible solution was found.
- FEASIBLE : A feasible solution was found, but we don't know if it's optimal.
- INFEASIBLE : The problem was proven infeasible.
- MODEL_INVALID : The given CpModelProto didn't pass the validation step. You can get a detailed error by calling ValidateCpModel(model_proto).
- UNKNOWN : The status of the model is unknown because no solution was found (or the problem was not proven INFEASIBLE) before something caused the solver to stop, such as a time limit, a memory limit, or a custom limit set by the user.



In [9]:
# display the first solution
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('x = %i' % solver.Value(x))
    print('y = %i' % solver.Value(y))
    print('z = %i' % solver.Value(z))
else:
    print('No solution found.')

x = 1
y = 0
z = 0


In [10]:
# Finding all solutions

# Add the solution printer
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count

# call the solver
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([x, y, z])
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True
# Solve.
status = solver.Solve(model, solution_printer)
print('Status = %s' % solver.StatusName(status))
print('Number of solutions found: %i' % solution_printer.solution_count())

x=1 y=2 z=0 
x=1 y=0 z=0 
x=2 y=0 z=0 
x=2 y=1 z=0 
x=2 y=1 z=1 
x=2 y=0 z=1 
x=1 y=0 z=1 
x=1 y=2 z=1 
x=1 y=2 z=2 
x=1 y=0 z=2 
x=2 y=0 z=2 
x=2 y=1 z=2 
x=0 y=1 z=2 
x=0 y=1 z=1 
x=0 y=1 z=0 
x=0 y=2 z=0 
x=0 y=2 z=1 
x=0 y=2 z=2 
Status = OPTIMAL
Number of solutions found: 18


### Example : finding an optimal solution

Maximize 2x + 2y + 3z subject to the following constraints:
```
x + 7⁄2 y + 3⁄2 z	≤	25
3x - 5y + 7z	≤	45
5x + 2y - 6z	≤	37
x, y, z	≥	0
x, y, z integers
```

In order to increase computational speed, the CP-SAT solver works over the integers. This means all constraints and the objective must have integer coefficients. In the above example, the first constraint doesn't meet this condition. To solve the problem, you must first transform the constraint by multiplying it by a sufficiently large integer to convert all the coefficients to integers. 

In [12]:
"""Simple solve."""
# Import the libraries
from ortools.sat.python import cp_model

"""Minimal CP-SAT example to showcase calling the solver."""
# Creates the model.
model = cp_model.CpModel()

# Creates the variables.
var_upper_bound = max(50, 45, 37)
x = model.NewIntVar(0, var_upper_bound, 'x')
y = model.NewIntVar(0, var_upper_bound, 'y')
z = model.NewIntVar(0, var_upper_bound, 'z')

# Creates the constraints.
model.Add(2 * x + 7 * y + 3 * z <= 50)
model.Add(3 * x - 5 * y + 7 * z <= 45)
model.Add(5 * x + 2 * y - 6 * z <= 37)

model.Maximize(2 * x + 2 * y + 3 * z)

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Maximum of objective function: {solver.ObjectiveValue()}\n')
    print(f'x = {solver.Value(x)}')
    print(f'y = {solver.Value(y)}')
    print(f'z = {solver.Value(z)}')
else:
    print('No solution found.')

# Statistics.
print('\nStatistics')
print(f'  status   : {solver.StatusName(status)}')
print(f'  conflicts: {solver.NumConflicts()}')
print(f'  branches : {solver.NumBranches()}')
print(f'  wall time: {solver.WallTime()} s')

Maximum of objective function: 35.0

x = 7
y = 3
z = 5

Statistics
  status   : OPTIMAL
  conflicts: 0
  branches : 0
  wall time: 0.0039381220000000005 s
